In [1]:
# Создание DataFrame и использование join

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import avg, max, asc, desc, col

spark = SparkSession.builder \
    .appName('schema') \
    .config('spark.master', 'spark://spark-master:7077') \
    .getOrCreate()

df1 = spark.createDataFrame([(1, 'Alice'),(2, 'Bob'),(3, 'Den')], ['id', 'name'])
df2 = spark.createDataFrame([(1, 2, 100), (2, 1, 200)], ['id', 'user_id', 'salary'])

df_join = df1.join(df2, df1.id == df2.user_id, how='left').select(df1.name, df2.salary)
df_join.show()

df1.show()

spark.stop()


+-----+------+
| name|salary|
+-----+------+
|Alice|   200|
|  Den|  NULL|
|  Bob|   100|
+-----+------+

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|  Den|
+---+-----+



In [1]:
# Запись данных в Parquet
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('Parquet-HDFS') \
    .config('spark.master', 'spark://spark-master:7077') \
    .config('spark.hadoop.fs.defaultFS', 'hdfs://hadoop-namenode:9000') \
    .getOrCreate()

path = 'hdfs://hadoop-namenode:9000/user/jovyan/parquet'

df1 = spark.createDataFrame([(1, 'Alice'),(2, 'Bob'),(3, 'Den')], ['id', 'name'])
df1.write.format('parquet').mode('overwrite').save(path)

df_parquet = spark.read.parquet(path)
df_parquet.show()

spark.stop()


+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|  Den|
+---+-----+

